# Mini-introducción a Scala #2: clases y colecciones
<!-- v 1.2 -- Paulo Villegas, 2020 -->

In [ ]:
// Which version of Scala?
scala.util.Properties.versionNumberString

## 1. Clases
### 1.1 Definición

La definición de clases en Scala es directa, y muy similar a Java
* `class` define una clase
* una clase tiene *campos* (*fields*) y *métodos* (*methods*); los dos juntos se llaman *miembros* (*members*)
  * los *campos* son variables de instancia, `val` o `var`
  * los métodos se definen mediante `def`, siguiendo la sintaxis de las funciones. Los parámetros que recibe un método son siempre `val` (es decir, no modificables)
* Un método que no devuelve nada (devuelve `Unit`) es un *procedimiento*: método útil sólo por sus efectos laterales ("side effects"). Los procedimientos pueden escribirse con una sintaxis abreviada (omitiendo `: Unit =`) pero se desaconseja por la posible confusión
* un objeto de una clase se instancia mediante `new`, y se asigna a una variable (de nuevo `var` o `val`)
* no hay constructor, los parámetros usados para la creación de la clase se definen directamente sobre la clase, y las operaciones necesrias durante la construcción son las contenidas en el cuerpo de la definición de la clase.
* se pueden definir *operadores* para la clase exactamente igual que los métodos

In [ ]:
/* Define a class */
class Point( xc: Int, yc: Int ) {

    // Class fields - we could abbreviate and put them directly as construction parameters
    var x: Int = xc
    var y: Int = yc
    
    // This method is a procedure (no return value, only side effects). Futhermore, it has no arguments
    def reset(): Unit = {
        x = 0
        y = 0
    }
    
    // This is also a procedure, but it takes two arguments.
    def move(dx: Int, dy: Int) {
        x = x + dx
        y = y + dy
    }
    
    // Here we define an operator, also as a procedure
    def +=(v: Int) : Unit = {
        x += v
        y += v
    }
    
    /* Overload the representation method */
    override def toString: String = s"($x,$y)";
}

In [ ]:
/* Create an instance */
val p = new Point( 10, 11 )

In [ ]:
/* and operate with it. This is a procedure, so no output */
p.move( 2, -1 )

In [ ]:
println( p )

In [ ]:
p.reset()
println(p)

In [ ]:
p += 2
println(p)

Las funciones o procedimientos sin argumentos se pueden llamar también omitiendo los paréntesis

In [ ]:
p.reset

println(p)

Los campos de un objeto pueden modificarse desde fuera, a no ser que los declaremos privados usando `private`. Obviamente siempre que no sean variables inmutables.

In [ ]:
p.x = 4

In [ ]:
p

### 1.2 apply
El método `apply` es especial, en el sentido de que es el que se ejecuta cuando se llama al objeto como si fuera una función

In [ ]:
// A class that computes the Euclidean distance to a reference point
class PointDistance(xc: Int, yc:Int) extends Point(xc, yc) {

    import scala.math.{sqrt, pow}
    
    def apply(x1: Int, y1: Int): Double = sqrt( (pow(x-x1,2) + pow(y-y1,2)).toDouble)
}

In [ ]:
var pd = new PointDistance(3, 4)

In [ ]:
// Distance from (7,8) to the reference point (3,4)
pd(7, 8)

### 1.3 Traits
Lo que en Java son `interfaces` (objetos que no definen clases, sino comportamiento para incluir en clases) en Scala se denominan `Traits`. A diferencia de una interfaz pura, un `Trait` puede _implementar_ comportamiento, además de definirlo.

In [ ]:
trait Proximity {
    def isNear(x: Any): Boolean
    def isFar(x: Any): Boolean = !isNear(x)
}

In [ ]:
// This will fail: we do not implement the whole trait
class Point( xc: Int, yc: Int ) extends Proximity {

    var x: Int = xc
    var y: Int = yc
    
    /* Overload the representation method */
    override def toString: String = s"($x,$y)";
}

In [ ]:
class ProxPoint( xc: Int, yc: Int ) extends Proximity {

    var x: Int = xc
    var y: Int = yc
    
    def isNear( obj: Any ) = obj.isInstanceOf[Point] &&
     (math.abs(obj.asInstanceOf[Point].x - x) +
      math.abs(obj.asInstanceOf[Point].y - y) < 5)

    override def toString: String = s"($x,$y)";
}

In [ ]:
val p = new ProxPoint( 10, 11 )

val p_near = new Point( 11, 12 )
val p_far  = new Point( 11, 20 )

In [ ]:
// is p_near near p?
val near = p.isNear( p_near )
println( s"IS NEAR: $near" )

// is p_near far from p?
println( s"IS FAR: ${p.isFar(p_near)}" )

### 1.3 Parametrización de tipos

Scala permite crear objetos genéricos parametrizando los tipos en su definición: `Type[T]`

In [ ]:
// Create a box of anything
class Box[T]( elem: T ) {

 val contents = elem
 
 def contains(obj: T) =  obj == contents
}

In [ ]:
// A box holding a string
val bs = new Box[String]("MyString")

In [ ]:
bs.contains( "MyString" )

In [ ]:
bs.contains( "notMyString" )

In [ ]:
// A box holding an integer
val bi = new Box[Int](17)

In [ ]:
bi.contains(17)

## 2. Colecciones

Scala predefine una cantidad de colecciones (similares a las existentes en otros lenguajes de programación) listas para ser usadas. La documentación oficial está en [Scala collections](http://docs.scala-lang.org/overviews/collections/introduction.html).

### 2.1 Secuencias

Los dos tipos de secuencia más importantes son `List` y `Array`
 * `List` es inmutable, y está orientada a la iteración
 * `Array` es mutable, y está optimizada para acceso arbitrario

In [ ]:
val l = List( 1, 2, 3, 4, 5 )

In [ ]:
// Acceso a un elemento. Para List, es O(n)
l(2)

In [ ]:
val a = Array( 1, 2, 3, 4, 5 )

In [ ]:
// Acceso a un elemento. Para Array, es O(1)
a(2)

Operaciones con listas

In [ ]:
l.reverse

In [ ]:
// Primer elemento
l.head

In [ ]:
// Resto de elementos
l.tail

In [ ]:
// suma el primer y el segundo elemento
def addHead( l: List[Int] ) = l.head + l.tail.head

In [ ]:
addHead(l)

En colecciones aparece una clase para representar la nada: `Nil`, que es una clase inmutable que representa una lista vacía

In [ ]:
Nil

### 2.2 Sets y Maps
Un set es una colección no ordenada de items, con implementación de operaciones de pertenencia y operaciones lógicas:

In [ ]:
val s = Set( "a", "b", "c", "d")

In [ ]:
s.contains( "c" )

In [ ]:
s & Set("a","c","f")

### 2.3 Ranges

`Ranges` son colecciones de elementos numéricos

In [ ]:
val r = Range(1,10)

In [ ]:
print(r)

Hay una sintaxis especial para definir rangos usando `to` o `until`

In [ ]:
1 to 10

In [ ]:
// define a range with a fractional increment, and convert the result to Array
(1.0 until 10.0 by 0.5).toArray

### 2.4 Tuplas
Una [tupla](https://en.wikibooks.org/wiki/Scala/Tuples) es una colección ordenada de elementos heterogéneos. Al ser heterogéneos, es una colección no iterable.

Se crea simplemente usando paréntesis.

In [ ]:
val oneTuple = (1, "some text", 2.0)

In [ ]:
oneTuple.getClass

La forma de acceder a los elementos de una tupla es mediante `._N`

In [ ]:
oneTuple._2

Una utilidad de las tuplas es para devolver múltiples valores de una función sin necesidad de tener que crear una clase específica.

In [ ]:
def plus( a:Int ) = { (a, a+1, a+2) }

val (v1,v2,v3) = plus( 5 )

In [ ]:
v2

### 2.5 Generadores

Los generadores son colecciones que producen sus elementos al iterarlas, no antes

In [ ]:
// Create a collection with all the files in the current directory
val filesHere = (new java.io.File(".")).listFiles

In [ ]:
// Define a generator with a filter over the collection
for (file <- filesHere if file.getName.endsWith(".ipynb"))
    println(file)

### 2.6 Operaciones de nivel superior sobre colecciones

Las colecciones de Scala definen métodos para aplicar operaciones a los elementos contenidos en ellos mediante funciones de nivel superior.

Por ejemplo, el método `map` es una función de alto nivel, que toma una función como argumento y la aplica a todos los elementos de la colección

In [ ]:
val numbers = List(1, 2, 3, 4)

In [ ]:
// We use here an anonymous function
numbers.map( x => 2 * x ) 

In [ ]:
// Same, but now with the shorthand using the underscore, 
// which is interpreted as a reference to an anonymous argument
numbers.map( 2 * _ ) 

In [ ]:
// Another example
numbers.filter( _ <= 2 )

Otra variante: el método `reduceLeft` aplica una función binaria (2 argumentos) de forma iterativa, hasta que todos los elementos de la colección han sido consumidos.

In [ ]:
numbers.reduceLeft( (a, b) => a + b )

In [ ]:
// Again, using the underscore shorthand (each underscore refers to one of the two parameters)
numbers.reduceLeft( _ + _ )